<a href="https://colab.research.google.com/github/ravipratap366/Complete-Deep-Learning/blob/master/PhotoGAUGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import cv2
from IPython.display import Image
import os 

In [2]:
#import utlis

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.841379297000003
GPU (s):
0.0389627669999868
GPU speedup over CPU: 72x


In [ ]:
#!unzip '/content/drive/MyDrive/OneDrive_2022-07-14.zip' -d '/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE'

In [ ]:
#Image('/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png')

In [ ]:

def getContours(img):
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur,cThr[0],cThr[1])
    kernel = np.ones((5,5))
    imgDial = cv2.dilate(imgCanny,kernel,iterations=3)
    imgThre = cv2.erode(imgDial,kernel,iterations=2)
    if showCanny:cv2.imshow('Canny',imgThre)
    contours,hiearchy = cv2.findContours(imgThre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    finalCountours = []
    for i in contours:
        area = cv2.contourArea(i)
        if area > minArea:
            peri = cv2.arcLength(i,True)
            approx = cv2.approxPolyDP(i,0.02*peri,True)
            bbox = cv2.boundingRect(approx)
            if filter > 0:
                if len(approx) == filter:
                    finalCountours.append([len(approx),area,approx,bbox,i])
            else:
                finalCountours.append([len(approx),area,approx,bbox,i])
    finalCountours = sorted(finalCountours,key = lambda x:x[1] ,reverse= True)
    if draw:
        for con in finalCountours:
            cv2.drawContours(img,con[4],-1,(0,0,255),3)
    return img, finalCountours

def reorder(myPoints):
    #print(myPoints.shape)
    myPointsNew = np.zeros_like(myPoints)
    myPoints = myPoints.reshape((4,2))
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

def warpImg (img,points,w,h,pad=20):
    # print(points)
    points =reorder(points)
    pts1 = np.float32(points)
    pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgWarp = cv2.warpPerspective(img,matrix,(w,h))
    imgWarp = imgWarp[pad:imgWarp.shape[0]-pad,pad:imgWarp.shape[1]-pad]
    return imgWarp

def findDis(pts1,pts2):
    return ((pts2[0]-pts1[0])**2 + (pts2[1]-pts1[1])**2)**0.5

In [ ]:
path = '/content/drive/MyDrive/assignment/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png'
img = cv2.imread(path)

In [ ]:
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [ ]:
#getContours('/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png')

In [ ]:


###################################
webcam = True
path = '/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png'
cap = cv2.VideoCapture(0)
cap.set(10,160)
cap.set(3,1920)
cap.set(4,1080)
scale = 3
wP = 210 *scale
hP= 297 *scale
###################################

while True:
    if webcam:success,img = cap.read()
    else: img = cv2.imread(path)

 

    imgContours , conts = getContours(img,minArea=50000,filter=4)
    if len(conts) != 0:
        biggest = conts[0][2]
        #print(biggest)
        imgWarp = warpImg(img, biggest, wP,hP)
        imgContours2, conts2 = getContours(imgWarp,
                                                 minArea=2000, filter=4,
                                                 cThr=[50,50],draw = False)
        if len(conts) != 0:
            for obj in conts2:
                cv2.polylines(imgContours2,[obj[2]],True,(0,255,0),2)
                nPoints = reorder(obj[2])
                nW = round((findDis(nPoints[0][0]//scale,nPoints[1][0]//scale)/10),1)
                nH = round((findDis(nPoints[0][0]//scale,nPoints[2][0]//scale)/10),1)
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[1][0][0], nPoints[1][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05)
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[2][0][0], nPoints[2][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05)
                x, y, w, h = obj[3]
                cv2.putText(imgContours2, '{}cm'.format(nW), (x + 30, y - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 255), 2)
                cv2.putText(imgContours2, '{}cm'.format(nH), (x - 70, y + h // 2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 255), 2)
        cv2.imshow('A4', imgContours2)

    img = cv2.resize(img,(0,0),None,0.5,0.5)
    cv2.imshow('Original',img)
    cv2.waitKey(1)

NameError: ignored

stitcher

In [ ]:
import numpy as np
import pandas as pd
import cv2
from IPython.display import Image
import os 
from google.colab.patches import cv2_imshow
import uuid   # Unique identifier


In [ ]:
cv2.__version__

'4.1.2'

In [ ]:
main_folder = '/content/drive/MyDrive/x'
my_folders = os.listdir(main_folder)
print(my_folders)

['3', '.ipynb_checkpoints']


In [ ]:
for folder in my_folders:
  path = main_folder +'/'+folder
  images=[]
  myList=os.listdir(path)
  print(f'total no of image detected {len(myList)}')
  for imgN in myList:
    curImg=cv2.imread(f'{path}/{imgN}')
    images.append(curImg)
  #print(len(images))
  stitcher = cv2.Stitcher.create() 
  (status,result) = stitcher.stitch(images)
  if (status == cv2.STITCHER_OK):
    print('Panorama Generated')
    cv2_imshow(result)


    IMAGES_PATH='/content/drive/MyDrive/panorama'
    # Naming out image path
    imgname = os.path.join(IMAGES_PATH,folder+str(uuid.uuid1())+'.png')

    # Writes out image to file 
    cv2.imwrite(imgname, result)
    print(imgname)
    print('Panorama saved')

    cv2.waitKey(1)
  else:
    print('panorama generation unsuccessful')

cv2.waitKey(0)



total no of image detected 3


###############

In [ ]:
path = '/content/drive/MyDrive/assignment/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png'
img = cv2.imread(path)

In [ ]:
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
imgCanny = cv2.Canny(imgBlur,50,100)
kernel = np.ones((5,5))
imgDial = cv2.dilate(imgCanny,kernel,iterations=3)
imgThre = cv2.erode(imgDial,kernel,iterations=2)

In [ ]:
#imgThre

In [5]:
#cv2_imshow(imgGray)

In [6]:
#cv2_imshow(imgBlur)


In [7]:
#cv2_imshow(img)


In [8]:
#cv2_imshow(imgThre)


In [ ]:
!pip install pyyaml==5.1
!pip show pyyaml  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 7.7 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=af517378d223f5dae1d3a1bc6f9f9b120b8b826e56e8fb8a368f998e5a3b120d
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Name: PyYAML
Version: 5.1
Summary: YAML parser and emitter for Python
Home-page: https://github.com/yaml/pyyaml
Author: Kirill Simonov
Author-email: xi@resolvent.net
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: PyDrive, fastai, distributed, coveralls, bokeh


In [ ]:
import tensorflow as tf
tf.__version__

'2.8.2'

In [ ]:
#!nvidia-smi
!nvcc --version #10.1
!python --version # 3.7.6

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Python 3.7.13


In [ ]:
!pip3 install torch==1.8.0 torchvision==0.9.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 763.5 MB 14 kB/s 
     |████████████████████████████████| 17.3 MB 68.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu113
    Uninstalling torchvision-0.13.0+cu113:
      Successfully uninstalled torchvision-0.13.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.8.0+cu101 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.8.0+cu10

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.8.0+cu101 True


In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git


Cloning into 'detectron2'...
remote: Enumerating objects: 14465, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 14465 (delta 14), reused 31 (delta 12), pack-reused 14425
Receiving objects: 100% (14465/14465), 5.93 MiB | 24.08 MiB/s, done.
Resolving deltas: 100% (10455/10455), done.


In [ ]:
%cd detectron2


/content/detectron2


In [ ]:
!pip install git+https://github.com/facebookresearch/fvcore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-yryz10bo
  Running command git clone -q https://github.com/facebookresearch/fvcore /tmp/pip-req-build-yryz10bo
     |████████████████████████████████| 42 kB 856 kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=65351 sha256=421e966f6bdc2c61ebbb1e896d92f004b2428785f694fb0e295f5312285d51ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-s0em6tgz/wheels/1c/80/2d/315cfb0174b5497db94b1a3c33b95db050f9172f1467171992
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=8a9dd557486165036fa18f846c45a5dbe192bd831dbc673f6dfaeccf2d852275
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath


In [ ]:
pip install cython; pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-req-build-kkbx003s
  Running command git clone -q https://github.com/philferriere/cocoapi.git /tmp/pip-req-build-kkbx003s
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=265201 sha256=9a93cf10145fb805d2c5cb2c5d7a51ab498c3f4fdb25f112f3fdee46e71c858a
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvddg7a_/wheels/6b/c6/c5/cb6da4cb793a6cb1ab91f6578d76c42686422127eb4dbcea94
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


In [ ]:
!pip install omegaconf


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.9 MB/s 
     |████████████████████████████████| 117 kB 17.2 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=19e52ed9dbefdcf6b918b28ee0cb68213483088f3806540f8dcc27688133e475
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


In [ ]:
!python setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/detectron2
copying detectron2/__init__.py -> build/lib.linux-x86_64-3.7/detectron2
creating build/lib.linux-x86_64-3.7/tools
copying tools/__init__.py -> build/lib.linux-x86_64-3.7/tools
copying tools/visualize_json_results.py -> build/lib.linux-x86_64-3.7/tools
copying tools/visualize_data.py -> build/lib.linux-x86_64-3.7/tools
copying tools/plain_train_net.py -> build/lib.linux-x86_64-3.7/tools
copying tools/analyze_model.py -> build/lib.linux-x86_64-3.7/tools
copying tools/benchmark.py -> build/lib.linux-x86_64-3.7/tools
copying tools/convert-torchvision-to-d2.py -> build/lib.linux-x86_64-3.7/tools
copying tools/train_net.py -> build/lib.linux-x86_64-3.7/tools
copying tools/lightning_train_net.py -> build/lib.linux-x86_64-3.7/tools
copying tools/lazyconfig_train_net.py -> build/lib.linux-x86_64-3.7/tools
creating build/lib.linux-x86_64-3.7/detectron2/projects
copying

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2


fatal: destination path 'detectron2' already exists and is not an empty directory.
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content/detectron2/detectron2


In [ ]:
!cd

In [ ]:
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2



ModuleNotFoundError: ignored